<!-- ![](./header.png) -->
<img src="./header.png",width=100%>

# The Photometric LSST Astronomical Time-series Classification Challenge (PLAsTiCC): code that runs the performance metric

*Alex Malz (NYU)*, *Renee Hlozek (U. Toronto)*, *Tarek Alam (UCL)*, *Anita Bahmanyar (U. Toronto)*, *Rahul Biswas (U. Stockholm)*, *Rafael Martinez-Galarza (Harvard)*, *Gautham Narayan (STScI)*

In [ ]:
import numpy as np
import pandas as pd

import matplotlib as mpl
mpl.use('Agg')
mpl.rcParams['text.usetex'] = False
mpl.rcParams['mathtext.rm'] = 'serif'
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['Times New Roman']
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['savefig.dpi'] = 250
mpl.rcParams['figure.dpi'] = 250
mpl.rcParams['savefig.format'] = 'pdf'
mpl.rcParams['savefig.bbox'] = 'tight'
import matplotlib.pyplot as plt
%matplotlib inline

# This is the code available on GitHub for calculating metrics,
# as well as performing other diagnostics on probability tables.

import proclam
from proclam import *

In [ ]:
# These are all functions for preprocessing Renee's data files.

def make_class_pairs(data_info_dict):
    for name in data_info_dict['names']:
        data_info_dict['class_pairs'][name] = [data_info_dict['classifications'][name], data_info_dict['truth_tables'][name]]
    return(data_info_dict['class_pairs'])
        
def make_file_locs(data_info_dict):
    names = data_info_dict['names']
    data_info_dict['dirname'] = topdir + data_info_dict['label'] + '/'
    for name in names:
        data_info_dict['classifications'][name] = '%s/predicted_prob_%s.csv'%(name, name)
        data_info_dict['truth_tables'][name] = '%s/truth_table_%s.csv'%(name, name)
    return data_info_dict

def process_strings(dataset, cc):
    loc = dataset['dirname']
    text = dataset['label'] + ' ' + dataset['names'][cc]
    return loc, text

def just_read_class_pairs(pair, dataset, cc):
    loc, text = process_strings(dataset, cc)
    clfile = pair[0]
    truthfile = pair[1]
    prob_mat = pd.read_csv(loc + clfile, delim_whitespace=True).values
    nobj = np.shape(prob_mat)[0]
    nclass = np.shape(prob_mat)[1]
    truth_values = pd.read_csv(loc + truthfile, delim_whitespace=True).values
    nobj_truth = np.shape(truth_values)[0]
    nclass_truth = np.shape(truth_values)[1]
    tvec = np.where(truth_values==1)[1]
    pmat = prob_mat
    return pmat, tvec

In [ ]:
# Renee gave me this data about which I know nothing.
mystery = {}
mystery['label'] = 'Unknown'
mystery['names'] = ['RandomForest', 'KNeighbors', 'MLPNeuralNet']
mystery['classifications'] = {}
mystery['truth_tables'] = {}
mystery['class_pairs'] = {}
mystery['probs'] = {}
mystery['truth'] = {}

In [ ]:
# We're just preprocessing Renee's data here.

topdir = '../examples/'
mystery = make_file_locs(mystery)
mystery['class_pairs'] = make_class_pairs(mystery)
for nm, name in enumerate(mystery['names']):
    probm, truthv = just_read_class_pairs(mystery['class_pairs'][name], mystery, nm)
    mystery['probs'][name] = probm
    mystery['truth'][name] = truthv
M_classes = np.shape(probm)[-1]

# we need the class labels in the dataset in a consistently sorted order 
# and will assume the weights of the weightvec correspond to this order
class_labels = sorted(np.unique(mystery['truth']['RandomForest']))

Method (Metric)
======

The log-loss is defined as
\begin{eqnarray*}
L &=& -\sum_{m=1}^{M}\frac{w_{m}}{N_{m}}\sum_{n=1}^{N_{m}}\ln[p_{n}(m | m)]
\end{eqnarray*}

We calculate the metric within each class $m$ by taking an average of its value $-\ln[p_{n}(m | m)]$ for each true member  $n$ of the class.  Then we weight the metrics for each class by an arbitrary weight $w_{m}$ and take a weighted average of the per-class metrics to produce a global scalar metric $L$.

In [ ]:
# This is how you run the metric with a random weight vector.

metric = 'LogLoss'
weightvec = np.ones(M_classes)
# dummy example for SNPhotCC demo data
special_classes = (1, 10, 11, 12)
# we should be using this for the PLAsTiCC data
# special_clases = (51, 99)
mask = np.array([True if classname in special_classes else False for classname in class_labels])
weightvec[mask] = 2
weightvec = weightvec / sum(weightvec)
name = np.random.choice(mystery['names'])
probm = mystery['probs'][name]
truthv = mystery['truth'][name]
LL = getattr(proclam.metrics, metric)()
val = LL.evaluate(prediction=probm, truth=truthv, averaging=weightvec)
print(name+' with weights '+str(weightvec)+' has '+metric+' = '+str(val))

Acknowledgments
===============

The DESC acknowledges ongoing support from the Institut National de Physique Nucleaire et de Physique des Particules in France; the Science & Technology Facilities Council in the United Kingdom; and the Department of Energy, the National Science Foundation, and the LSST Corporation in the United States.

DESC uses resources of the IN2P3 Computing Center (CC-IN2P3--Lyon/Villeurbanne - France) funded by the Centre National de la Recherche Scientifique; the National Energy Research Scientific Computing Center, a DOE Office of Science User Facility supported by the Office of Science of the U.S. Department of Energy under Contract No. DE-AC02-05CH11231; STFC DiRAC HPC Facilities, funded by UK BIS National E-infrastructure capital grants; and the UK particle physics grid, supported by the GridPP Collaboration.

This work was performed in part under DOE Contract DE-AC02-76SF00515.

Contributions
=======

Alex Malz: conceptualization, data curation, formal analysis, investigation, methodology, project administration, software, supervision, validation, visualization, writing - original draft

Renee Hlozek: data curation, formal analysis, funding acquisition, investigation, project administration, software, supervision, validation, visualization, writing - original draft

Tarek Alam: investigation, software, validation

Anita Bahmanyar: formal analysis, investigation, methodology, software, writing - original draft

Rahul Biswas: conceptualization, methodology, software

Rafael Martinez-Galarza: data curation, software, visualization

Gautham Narayan: data curation, formal analysis